<a href="https://colab.research.google.com/github/atharabbas993/Solar_Panels_Dust_Predictor/blob/main/Solar_Panel_Dust_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import Libraries

In [ ]:
import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential

from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
!kaggle datasets download -d hemanthsai7/solar-panel-dust-detection

#Split the Data

In [ ]:
!pip install split-folders

In [ ]:
from zipfile import ZipFile

with ZipFile('solar-panel-dust-detection.zip', 'r') as zipObj:
   zipObj.extractall()


In [ ]:
import splitfolders

#splits the data into 80% for training, 10% for validation, and 10% for testing.

splitfolders.ratio("/content/Detect_solar_dust", output="output", seed=1337, ratio=(.8, 0.1,0.1),group_prefix=None)


#2. Data Preparation

In [ ]:
# Set paths to the dataset

#Train data folder Directory

train_dir = '/content/output/train'

#Validation_Data_Directory

validation_dir = '/content/output/val'

#Test_Data_Directory

test_dir = '/content/output/test'




#Image Data Generator for augmentation

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)


#____________Train_Data__________________

# Load images from directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images
    batch_size=32,
    class_mode='binary'     # Binary classification
)

#___________Validation_Data__________________

validation_data = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

#_____________Test_Data________________________

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


#3. Build the CNN Model

In [ ]:
model = Sequential()

#_________Convalutional_Layers_______

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#______Flatten_Layer___

model.add(Flatten())

#______Fully_Connected_Layers______

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))


#Model Architecture

In [ ]:
from keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True)

# Model Summary

In [ ]:
#Shows the model's layer details and parameters.

model.summary()

#4. Compile the Model

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#Implement Early_stopping

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=3,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

#5. Train the Model


In [ ]:

history = model.fit(train_data, epochs=30, validation_data=validation_data, callbacks=[early_stopping])


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


#8. Visualize Training Results (Optional)

In [ ]:
#_____________Visualize_Training_Results_________________

import matplotlib.pyplot as plt

# Plot accuracy and loss over epochs
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

#1. Visualize Accuracy and Loss

In [ ]:
import matplotlib.pyplot as plt

# Function to plot accuracy and loss
def plot_training_history(history):
    # Plot training and validation accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Model Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Model Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the function
plot_training_history(history)


#2. Confusion Matrix for Test Results

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# Get true labels and predictions
test_data.reset()  # Reset test data generator to prevent order mismatch
y_true = test_data.classes
y_pred = (model.predict(test_data) > 0.5).astype("int32").flatten()

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_data.class_indices.keys())

# Plot confusion matrix
plt.figure(figsize=(8, 8))
disp.plot(cmap='Blues', values_format='d')
plt.title('Confusion Matrix')
plt.show()

# Print classification report
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=test_data.class_indices.keys()))


#3. ROC Curve for Binary Classification

In [ ]:
from sklearn.metrics import roc_curve, auc

# Get probabilities for ROC curve
y_probs = model.predict(test_data).flatten()  # Get predicted probabilities
fpr, tpr, thresholds = roc_curve(y_true, y_probs)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random Guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()


#4. Precision-Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, _ = precision_recall_curve(y_true, y_probs)
avg_precision = average_precision_score(y_true, y_probs)

# Plot Precision-Recall curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'Precision-Recall Curve (AP = {avg_precision:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()


#5. Histogram of Predicted Probabilities

In [ ]:
import seaborn as sns

# Get predicted probabilities
y_probs = model.predict(test_data).flatten()

# Plot histogram
plt.figure(figsize=(8, 6))
sns.histplot(y_probs, kde=True, bins=20, color='blue')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Histogram of Predicted Probabilities')
plt.grid()
plt.show()


#6. Learning Rate Schedule

In [ ]:
# Assuming you logged learning rates during training
learning_rates = [0.001, 0.0009, 0.0008, 0.0007, 0.0006]  # Example values

# Plot learning rate over epochs
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(learning_rates) + 1), learning_rates, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedule')
plt.grid()
plt.show()


#7. Training Time Per Epoch

In [ ]:
import time

# If you tracked training time during each epoch:
epoch_times = [5.2, 4.8, 5.1, 5.0, 5.3]  # Example times in seconds for each epoch

# Plot training time per epoch
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(epoch_times) + 1), epoch_times, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Time (seconds)')
plt.title('Training Time Per Epoch')
plt.grid()
plt.show()


#8. Class Distribution in the Dataset

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Count class instances
class_counts = pd.DataFrame({
    'Class': ['Clean', 'Dusty'],
    'Count': [len(os.listdir('/content/output/train/Clean')), len(os.listdir('/content/output/train/Dusty'))]
})

# Bar plot of class distribution
plt.figure(figsize=(8, 6))
sns.barplot(x='Class', y='Count', data=class_counts, palette='coolwarm')
plt.xlabel('Class')
plt.ylabel('Number of Images')
plt.title('Class Distribution in Dataset')
plt.show()


#9. Model Performance Over Epochs with Log Scale

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.yscale('log')
plt.xlabel('Epochs')
plt.ylabel('Loss (Log Scale)')
plt.title('Loss Over Epochs (Log Scale)')
plt.legend()
plt.show()


#7. Save the Model

In [ ]:
#_____________Save_Model______________

model.save('solar_panel_classifier.h5')


# 8.Diploy the Model

In [ ]:
#________________Diploy the Model_____________________

from tensorflow.keras.models import load_model
model = load_model('solar_panel_classifier.h5')

# Make a prediction
import numpy as np
from tensorflow.keras.preprocessing import image

# Load a single image for prediction
img = image.load_img('/content/output/test/Dusty/Imgdirty_1054_1.jpg', target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_array)
if prediction[0][0] > 0.5:
    print("The solar panel is clean.")
else:
    print("The solar panel is dusty.")
